# How does search work?

In [ ]:
from laminci.db import setup_local_test_postgres
pgurl = setup_local_test_postgres()
!lamin init --name benchmark_search --db {pgurl} --schema bionty --storage ./benchmark_search

Here we show how to perform text search on `Record` and evaluate some search queries for the {class}`bionty.CellType` ontology.

In [ ]:
import lamindb as ln
import bionty as bt

SEARCH_QUERIES_EXACT = ("t cell", "stem cell", "b cell", "regulatory B cell", "Be2 cell", "adipocyte")
SEARCH_QUERIES_CONTAINS = ("t cel", "t-cel", "neural", "kidney", "kidne")
TOP_N = 20

bt.CellType.import_source()

## Search the registry

In [ ]:
for query in SEARCH_QUERIES_EXACT:
    print("Query:", query)
    qs = bt.CellType.search(query)
    display(qs.df())
    
    assert query.lower() == qs[0].name.lower()

In [ ]:
for query in SEARCH_QUERIES_CONTAINS:
    print("Query:", query)
    qs = bt.CellType.search(query)
    display(qs.df())

    top_record = qs[0]
    query = query.lower()
    assert query in top_record.name.lower() or query in top_record.synonyms.lower()

## Search the public ontology

In [ ]:
ct_public = bt.CellType.public()

df = ct_public.search("b cell", limit=20)
assert df.iloc[0]["name"] == "B cell"
df

In [ ]:
!docker stop pgtest && docker rm pgtest
!lamin delete --force benchmark_search